In [ ]:
%reset -f
import glob, os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Normal
from scipy.io import loadmat
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def load_hums_data(folder_path):
    data_list = []
    filenames = sorted(glob.glob(os.path.join(folder_path, '*.mat')))
    for file in filenames:
        mat = loadmat(file)
        if 'xr' in mat:
            xr = mat['xr'].astype(np.float32).flatten()
            data_list.append(xr)
    return data_list, filenames

train_seqs, train_files = load_hums_data('./data/AIRL/training_until20')
test_seqs, test_files = load_hums_data('./data/AIRL/testAIRL_from21')

# Normalize
all_train = np.concatenate(train_seqs)
mean_train, std_train = all_train.mean(), all_train.std()
train_seqs = [(seq - mean_train) / (std_train + 1e-9) for seq in train_seqs]
test_seqs = [(seq - mean_train) / (std_train + 1e-9) for seq in test_seqs]

# Create transitions
def create_transitions(seqs):
    s, a, sp = [], [], []
    for seq in seqs:
        s.extend(seq[:-1])
        a.extend(seq[1:])
        sp.extend(seq[1:])
    return torch.tensor(s).unsqueeze(-1).to(device), torch.tensor(a).unsqueeze(-1).to(device), torch.tensor(sp).unsqueeze(-1).to(device)



s_train, a_train, sp_train = create_transitions(train_seqs)


def save_scores(fname, file_list, score_list):
    arr = np.empty(len(file_list), dtype=object)   
    arr[:] = list(zip(file_list, score_list))
    np.save(fname, arr, allow_pickle=True)         


class RewardNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(2,64),nn.ReLU(),nn.Linear(64,1))
    def forward(self, s,a):
        return self.net(torch.cat([s,a], dim=1))

class ValueNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(1,64),nn.ReLU(),nn.Linear(64,1))
    def forward(self,s):
        return self.net(s)

class PolicyNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Sequential(nn.Linear(1,64), nn.ReLU())
        self.mean = nn.Linear(64,1)
        
        self.log_std = nn.Parameter(torch.ones(1) * -1.0)
    def forward(self, s):
        x = self.fc(s)
        std = torch.clamp(self.log_std.exp(), min=1e-3)   # avoid exact zero
        return self.mean(x), std

    def sample(self, s):
        mu, std = self.forward(s)
        dist = Normal(mu, std)
        a = dist.rsample()
        logp = dist.log_prob(a)
        return a, logp

# Instantiate models
reward_net = RewardNet().to(device)
value_net = ValueNet().to(device)
policy_net = PolicyNet().to(device)

opt_disc = optim.Adam(list(reward_net.parameters())+list(value_net.parameters()),lr=1e-3)
opt_policy = optim.Adam(policy_net.parameters(),lr=1e-3)

gamma             = 0.99
beta              = 0.01       
batch_size        = 128
episodes_per_epoch= 50
steps_per_episode = 50
num_epochs        = 300


for epoch in range(num_epochs):
    # Discriminator (AIRL)
    idx     = torch.randint(0, len(s_train), (batch_size,))
    s_exp, a_exp, sp_exp = s_train[idx], a_train[idx], sp_train[idx]
    s_fake, _ = s_train[idx], None
    a_fake, _ = policy_net.sample(s_fake)
    sp_fake   = a_fake

    r_exp     = reward_net(s_exp, a_exp)
    r_fake    = reward_net(s_fake, a_fake)
    V_se, V_spe = value_net(s_exp),  value_net(sp_exp)
    V_sf, V_spf = value_net(s_fake),  value_net(sp_fake)

    D_exp     = torch.sigmoid(r_exp + gamma*V_spe - V_se)
    D_fake    = torch.sigmoid(r_fake + gamma*V_spf - V_sf)

    loss_disc = - (torch.log(D_exp + 1e-9).mean()
                  + torch.log(1 - D_fake + 1e-9).mean())
    opt_disc.zero_grad()
    loss_disc.backward()
    opt_disc.step()

    # Policy
    policy_loss_total = 0.0
    for ep in range(episodes_per_epoch):
        idx = torch.randint(0, len(s_train), (1,))
        s   = s_train[idx]

        logps, rewards, entropies = [], [], []
        for step in range(steps_per_episode):
            a, logp = policy_net.sample(s)
            mu, std = policy_net.forward(s)
            dist     = Normal(mu, std)

            entropies.append(dist.entropy().mean())

            sp = a 
            with torch.no_grad():
                r = reward_net(s, a).detach()

            logps.append(logp)
            rewards.append(r)
  
        returns = []
        G = 0
        for r in reversed(rewards):
            G = r + gamma * G
            returns.insert(0, G)
        returns = torch.stack(returns)
        returns = (returns - returns.mean()) / (returns.std() + 1e-9)

        logps = torch.stack(logps).squeeze()   # shape [steps]
        ent   = torch.stack(entropies).mean()

        policy_loss_ep = -(logps * returns.detach()).mean() - beta * ent
        policy_loss_total += policy_loss_ep


    policy_loss_total /= episodes_per_epoch
    opt_policy.zero_grad()
    policy_loss_total.backward()
    opt_policy.step()

    if epoch % 10 == 0:
        print(
            f"Epoch {epoch:03d}  "
            f"Disc Loss={loss_disc:.4f}  "
            f"log_std={policy_net.log_std.item():+.3f}"
        )

In [ ]:
thresholds = {}
with torch.no_grad():
    D_train = torch.sigmoid(reward_net(s_train, a_train) + gamma * value_net(sp_train) - value_net(s_train))
    anom_train = (1 - D_train).cpu().numpy()

    file_ptr = 0
    file_level_scores = []
    for seq, fname in zip(train_seqs, train_files):
        n = len(seq) - 1
        score = anom_train[file_ptr : file_ptr + n].max()  # or .mean()
        file_level_scores.append(score)
        file_ptr += n

    save_scores(
        "train_anomaly_scores_sensorRF2.npy",
        train_files,
        file_level_scores
    )
    
    thresholds['max'] = anom_train.max()
    std_error = anom_train.std() / np.sqrt(len(anom_train))
    thresholds['max_minus_stderr'] = anom_train.max() - std_error
    n_samples = len(anom_train)
    std_error = anom_train.std() / np.sqrt(n_samples)
    thresholds['max_minus_2sterror'] = anom_train.max() - 2*std_error

    thresholds['manual_075'] = 0.78

test_scores = []
for seq, fname in zip(test_seqs, test_files):
    s, a, sp = create_transitions([seq])
    with torch.no_grad():
        D_test = torch.sigmoid(reward_net(s, a) + gamma * value_net(sp) - value_net(s))
        anom_score = (1 - D_test).max().item()
    test_scores.append((fname, anom_score))

save_scores("test_anomaly_scores_sensorRF2.npy",
            [f for f, _ in test_scores],
            [s for _, s in test_scores])

for method, thresh in thresholds.items():
    print(f"Threshold ({method}): {thresh:.4f}")
    anomalous_files = [f for f, s in test_scores if s > thresh]
    if anomalous_files:
        print("Earliest Anomalous File:", anomalous_files[0])
    else:
        print("No anomalies detected.")

ranked = sorted(test_scores, key=lambda x: x[1], reverse=True)
for fname, score in ranked[:]:
    print(f"{fname}: {score:.4f}")